In [ ]:
import json
import torch
import pdb
import pyarrow.feather as feather
from typing import List, Tuple, Any, Dict
from PIL import Image
import numpy as np
import pandas as pd
import pickle

from pathlib import Path
from .common import get_split, INTERPOLATION, get_view_matrix, get_pose
from .transforms import Sample, LoadDataTransform, SaveDataTransform

from av2.datasets.sensor.av2_sensor_dataloader import AV2SensorDataLoader
from av2.datasets.sensor.sensor_dataloader import SensorDataloader
from av2.datasets.sensor.constants import RingCameras, AnnotationCategories
from av2.geometry.camera.pinhole_camera import PinholeCamera
from av2.map.map_api import ArgoverseStaticMap, DrivableAreaMapLayer
from av2.geometry.se3 import SE3
from av2.geometry.geometry import compute_interior_points_mask
from av2.rendering.rasterize import xyz_to_bev
from av2.structures.cuboid import CuboidList
from av2.utils.io import read_city_SE3_ego, read_img
import random


STATIC = [
    AnnotationCategories.BOLLARD ,
    AnnotationCategories.CONSTRUCTION_BARREL ,
    AnnotationCategories.CONSTRUCTION_CONE ,
    AnnotationCategories.MOBILE_PEDESTRIAN_CROSSING_SIGN ,
    AnnotationCategories.SIGN ,
    AnnotationCategories.STOP_SIGN ,
    AnnotationCategories.TRAFFIC_LIGHT_TRAILER ,
    AnnotationCategories.MOBILE_PEDESTRIAN_CROSSING_SIGN ,

          ]
DYNAMIC = [
    AnnotationCategories.ARTICULATED_BUS ,
    AnnotationCategories.BICYCLE ,
    AnnotationCategories.ANIMAL ,
    AnnotationCategories.BICYCLIST ,
    AnnotationCategories.BOX_TRUCK ,
    AnnotationCategories.BUS ,
    AnnotationCategories.DOG ,
    AnnotationCategories.LARGE_VEHICLE ,
    AnnotationCategories.MESSAGE_BOARD_TRAILER ,
    AnnotationCategories.MOTORCYCLE ,
    AnnotationCategories.MOTORCYCLIST ,
    AnnotationCategories.OFFICIAL_SIGNALER ,
    AnnotationCategories.PEDESTRIAN ,
    AnnotationCategories.BICYCLIST ,
    AnnotationCategories.RAILED_VEHICLE ,
    AnnotationCategories.REGULAR_VEHICLE ,
    AnnotationCategories.SCHOOL_BUS ,
    AnnotationCategories.STROLLER ,
    AnnotationCategories.TRUCK ,
    AnnotationCategories.TRUCK_CAB ,
    AnnotationCategories.VEHICULAR_TRAILER ,
    AnnotationCategories.WHEELCHAIR ,
    AnnotationCategories.WHEELED_RIDER ,
    AnnotationCategories.WHEELED_DEVICE ,
          ]
NUM_CLASSES = len(STATIC) + len(DYNAMIC)

In [17]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)
sys.path.append('/srv/share2/apatni30/raster-net/comparison/cross_view_transformers/cross_view_transformer/')


In [18]:
from argoverse2_dataset import Argoverse2Dataset

ImportError: attempted relative import with no known parent package